In [1]:
import jax.numpy as jnp
from jax.ad_checkpoint import print_saved_residuals
import jax

In [2]:
def chain_compose(funs):
  def f(x):
    for fun in funs:
      x = fun(x)
  return f

f = chain_compose([jnp.sin] * 8)
print_saved_residuals(f, 3.)

In [3]:


f = chain_compose([jnp.sin] * 8 + [jnp.cos] * 8)
print_saved_residuals(f, 3.)

In [4]:
def recursive_checkpoint(funs):
  if len(funs) == 1:
    return funs[0]
  elif len(funs) == 2:
    f1, f2 = funs
    return lambda x: f1(f2(x))
  else:
    f1 = recursive_checkpoint(funs[:len(funs)//2])
    f2 = recursive_checkpoint(funs[len(funs)//2:])
    return lambda x: f1(jax.checkpoint(f2)(x))

In [5]:
f = recursive_checkpoint([jnp.sin] * 8 + [jnp.cos] * 8)
print_saved_residuals(f, 3.)

f32[] from the argument x
f32[] output of cos from /tmp/ipykernel_7673/1943107544.py:6 (<lambda>)
f32[] output of sin from /tmp/ipykernel_7673/1943107544.py:6 (<lambda>)
f32[] output of cos from /tmp/ipykernel_7673/1943107544.py:6 (<lambda>)
f32[] output of cos from /tmp/ipykernel_7673/1943107544.py:6 (<lambda>)


In [41]:
import numpy as np
def dummy_recursive_checkpoint(funs):
  if len(funs) == 1:
    #return funs[0]
    #print("END (len=1)", funs[0])
    return funs[0]
  elif len(funs) == 2:
    f1, f2 = funs
    #return lambda x: "A: f1(f2(x))"
    #print(f1+" "+f2)
    #return f1+" "+f2
    return f1+" "+f2
    #return f1,f2
  else:
    A = funs[:len(funs)//2]
    B = funs[len(funs)//2:]
    f1 = dummy_recursive_checkpoint(A)
    f2 = dummy_recursive_checkpoint(B)
    #print(f1+" ["+f2+"]")
    return f1+" ("+f2+")"
    
    #return lambda x: "B: f1(f2(x))"

In [50]:

def display_recursive_checkpoint(K):
    inlist = [f"f{i}" for i in range(K,0,-1)]
    #print(inlist)
    return dummy_recursive_checkpoint(inlist)


In [51]:
display_recursive_checkpoint(8)


'f8 f7 (f6 f5) (f4 f3 (f2 f1))'

In [54]:
display_recursive_checkpoint(16)

'f16 f15 (f14 f13) (f12 f11 (f10 f9)) (f8 f7 (f6 f5) (f4 f3 (f2 f1)))'